# Testing purpouses only

This notebook does not belong to the project pipeline, its use is only intended for single component experimentation.

**Note:** this notebook should be executed on a platform with available video devices mounted. wsl2 cannot access windows mounts by default.

In [ ]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt

In [ ]:
cap = cv2.VideoCapture(0)
face_detector = YOLO("Models/yolov11s-face.pt")
drowsiness_detector = YOLO("Models/best.pt")

In [ ]:
ret, frame = cap.read()
cap.release()

if ret:
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    plt.imshow(frame_rgb)
    plt.axis("off")
    plt.show()
else:
    print("Image capture failed.")

In [ ]:
results = face_detector(frame_rgb)

cropped_faces = []

for result in results:
    annotated_frame = result.plot()
    plt.imshow(annotated_frame)
    plt.axis("off")
    plt.show()

In [ ]:
cropped_faces = []
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cropped_face = frame_rgb[y1:y2, x1:x2]
        cropped_faces.append(cropped_face)
        plt.imshow(cropped_face)
        plt.axis("off")
        plt.show()

In [ ]:
results = drowsiness_detector(cropped_faces)

labels = ["Drowsy", "Non drowsy"]

n = len(results)
fig, axes = plt.subplots(1, n, figsize=(3 * n, 3))
if n == 1:
    axes = [axes]

for ax, element, img in zip(axes, results, cropped_faces):
    top1 = element.probs.top1
    prob_val = float(top1)

    title = f"{labels[0]} ({1 - prob_val:.2f})" if prob_val < 0.5 else f"{labels[1]} ({prob_val:.2f})"
    ax.imshow(img)
    ax.set_title(title)
    ax.axis("off")

plt.tight_layout()
plt.show()